In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "Salesforce/codet5p-220m"  # or try starcoderbase-1b
tokenizer = AutoTokenizer.from_pretrained(model_name, local_files_only=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, local_files_only=False)


In [3]:
sample = "func add(a int, b int) int { return a + b }"
tokens = tokenizer(sample, return_tensors="pt")
print(tokens.input_ids.shape)  # Should be (1, N)


torch.Size([1, 18])


In [5]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="../data/check_for_2015.jsonl", split="train")
dataset = dataset.train_test_split(test_size=0.2)

print(dataset["train"][0])


Generating train split: 3020 examples [00:01, 2673.48 examples/s]

{'instruction': 'Classify the Go code as Vulnerable or Secure with CWE ID', 'input': 'package application import ( "bufio" "encoding/json" "errors" "fmt" "reflect" "strings" "time" "github.com/Masterminds/semver" jsonpatch "github.com/evanphx/json-patch" log "github.com/sirupsen/logrus" "golang.org/x/net/context" "google.golang.org/grpc/codes" "google.golang.org/grpc/status" v1 "k8s.io/api/core/v1" apierr "k8s.io/apimachinery/pkg/api/errors" metav1 "k8s.io/apimachinery/pkg/apis/meta/v1" "k8s.io/apimachinery/pkg/apis/meta/v1/unstructured" "k8s.io/apimachinery/pkg/fields" "k8s.io/apimachinery/pkg/types" "k8s.io/apimachinery/pkg/watch" "k8s.io/client-go/kubernetes" "k8s.io/client-go/rest" "k8s.io/utils/pointer" "github.com/argoproj/argo-cd/pkg/apiclient/application" "github.com/argoproj/argo-cd/pkg/apis/application/v1alpha1" appv1 "github.com/argoproj/argo-cd/pkg/apis/application/v1alpha1" appclientset "github.com/argoproj/argo-cd/pkg/client/clientset/versioned" "github.com/argoproj/argo-

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5p-220m")

def preprocess(examples):
    inputs = [f"{i} {c}" for i, c in zip(examples["instruction"], examples["input"])]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    labels = tokenizer(examples["output"], max_length=64, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_ds = dataset.map(preprocess, batched=True)

Map: 100%|██████████| 604/604 [00:00<00:00, 1196.20 examples/s]


In [1]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5p-220m")

training_args = Seq2SeqTrainingArguments(
    output_dir="./models/codet5p-go-cwe",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=True  # set to True if on GPU with float16 support
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer
)

trainer.train()


/vol/home/s3905020/Desktop/thesis/slm-cwe-go/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


RuntimeError: Failed to import transformers.trainer_seq2seq because of the following error (look up to see its traceback):
cannot import name 'EncoderDecoderCache' from 'transformers' (/vol/home/s3905020/Desktop/thesis/slm-cwe-go/lib/python3.10/site-packages/transformers/__init__.py)